In [2]:
!pip install peakutils

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from peakutils import baseline
from sklearn import preprocessing
from scipy.signal import savgol_filter
from peakutils import indexes
from scipy.signal import find_peaks as fp
import numpy as np
import os
import pathlib

fileal1 = os.listdir('/content/drive/MyDrive/testml/Alanine_1')
fileal2 = os.listdir('/content/drive/MyDrive/testml/Alanine_2')
fileal3 = os.listdir('/content/drive/MyDrive/testml/Alanine_3')
filegl1 = os.listdir('/content/drive/MyDrive/testml/Glutamine_1')
filegl2 = os.listdir('/content/drive/MyDrive/testml/Glutamine_2')
filegl3 = os.listdir('/content/drive/MyDrive/testml/Glutamine_3')

In [5]:
# создаю датафрейм для усреднённого аланина
filepath = os.path.join('/content/drive/MyDrive/testml/Alanine_1/', fileal1[0])
temp = pd.read_csv(filepath, delimiter='\t', header = None)
temp.columns = ['x', 'y']
# print(temp)
for i in range(1, len(fileal1)):
  filepath = os.path.join('/content/drive/MyDrive/testml/Alanine_1/', fileal1[i])
  df = pd.read_csv(filepath, delimiter='\t', header = None)
  df.columns = ['x', 'y']
  for j in range(len(df)):
    temp.at[j, 'y'] = temp.iloc[j, 1] + df.iloc[j, 1]
for i in range(len(fileal2)):
  filepath = os.path.join('/content/drive/MyDrive/testml/Alanine_2/', fileal2[i])
  df = pd.read_csv(filepath, delimiter='\t', header = None)
  df.columns = ['x', 'y']
  for j in range(len(df)):
    temp.at[j, 'y'] = temp.iloc[j, 1] + df.iloc[j, 1]
for i in range(len(fileal3)):
  filepath = os.path.join('/content/drive/MyDrive/testml/Alanine_3/', fileal3[i])
  df = pd.read_csv(filepath, delimiter='\t', header = None)
  df.columns = ['x', 'y']
  for j in range(len(df)):
    temp.at[j, 'y'] = temp.iloc[j, 1] + df.iloc[j, 1]
    temp.at[j, 'y'] = temp.iloc[j, 1] / (len(fileal1) + len(fileal2) + len(fileal3))
# print(temp)

In [6]:
# высчитываю усреднённые пики у аланина, добавляю в сет с частотами пиков
scaler = preprocessing.MinMaxScaler()
names = temp.columns
d = scaler.fit_transform(temp)
df_scaler = pd.DataFrame(d, columns = names)

deg = 80
bl = baseline(df_scaler.y, deg = deg)

wl = 5
po = 1
df_filt = savgol_filter(df_scaler.y - bl, window_length = wl, polyorder = po)

t = {'x': temp.x, 'y': df_filt}
df_final = pd.DataFrame(t, columns = names)

h = 0.1
prom = 0.12
dist = None
p, _ = fp(df_final.y, height = h, prominence = prom, distance = dist)
mas = [0]*len(p)
for i in range(len(p)):
  mas[i] = df_final.iloc[p[i], 0]
tn = set(mas)

In [7]:
# создаю датафрейм усреднённого глутамина
filepath = os.path.join('/content/drive/MyDrive/testml/Glutamine_1/', filegl1[0])
temp = pd.read_csv(filepath, delimiter='\t', header = None)
temp.columns = ['x', 'y']
for i in range(1, len(filegl1)):
  filepath = os.path.join('/content/drive/MyDrive/testml/Glutamine_1/', filegl1[i])
  df = pd.read_csv(filepath, delimiter='\t', header = None)
  df.columns = ['x', 'y']
  for j in range(len(df)):
    temp.at[j, 'y'] = temp.iloc[j, 1] + df.iloc[j, 1]
for i in range(len(filegl2)):
  filepath = os.path.join('/content/drive/MyDrive/testml/Glutamine_2/', filegl2[i])
  df = pd.read_csv(filepath, delimiter='\t', header = None)
  df.columns = ['x', 'y']
  for j in range(len(df)):
    temp.at[j, 'y'] = temp.iloc[j, 1] + df.iloc[j, 1]
for i in range(len(filegl3)):
  filepath = os.path.join('/content/drive/MyDrive/testml/Glutamine_3/', filegl3[i])
  df = pd.read_csv(filepath, delimiter='\t', header = None)
  df.columns = ['x', 'y']
  for j in range(len(df)):
    temp.at[j, 'y'] = temp.iloc[j, 1] + df.iloc[j, 1]
    temp.at[j, 'y'] = temp.iloc[j, 1] / (len(filegl1) + len(filegl2) + len(filegl3))
# print(temp)

In [8]:
# высчитываю пики у усреднённого глутамина, добавляю частоты пиков в сет
scaler = preprocessing.MinMaxScaler()
names = temp.columns
d = scaler.fit_transform(temp)
df_scaler = pd.DataFrame(d, columns = names)

deg = 50
bl = baseline(df_scaler.y, deg = deg)

wl = 5
po = 1
df_filt = savgol_filter(df_scaler.y - bl, window_length = wl, polyorder = po)

t = {'x': temp.x, 'y': df_filt}
df_final = pd.DataFrame(t, columns = names)

h = 0.1
prom = 0.12
dist = None
p, _ = fp(df_final.y, height = h, prominence = prom, distance = dist)
for i in range(len(p)):
  tn.add(df_final.iloc[p[i], 0])

In [9]:
# создаю пустой главный датафрейм на нужное количество строк (аминокислот) и столбцами с названиями частот пиков у аланина и глутамина и "y" (1 -- аланин, 0 -- глутамин)
k = len(fileal1) + len(fileal2) + len(fileal3) + len(filegl1) + len(filegl2) + len(filegl3)
arr = list(tn)
arr.sort()
arr = arr[::-1]
maindf = pd.DataFrame(0, index = range(k), columns = ['y', *list(arr)])

In [10]:
# заполняю главный датафрейм, обрабатывая каждый спектр
for i in range(len(fileal1)):
  filepath = os.path.join('/content/drive/MyDrive/testml/Alanine_1/', fileal1[i])
  df = pd.read_csv(filepath, delimiter='\t', header = None)
  df.columns = ['x', 'y']
  scaler = preprocessing.MinMaxScaler()
  names = df.columns
  d = scaler.fit_transform(df)
  df_scaler = pd.DataFrame(d, columns = names)

  deg = 80
  bl = baseline(df_scaler.y, deg = deg)

  wl = 5
  po = 1
  df_filt = savgol_filter(df_scaler.y - bl, window_length = wl, polyorder = po)

  t = {'x': df.x, 'y': df_filt}
  df_final = pd.DataFrame(t, columns = names)
  pd.set_option('display.max_rows', None)
  maindf.at[i, 'y'] = 1
  for j in range(len(arr)):
    maindf.iloc[i, (j+1)] = df_final[df_final['x']==arr[j]].iloc[0, 1]

for i in range(len(fileal2)):
  filepath = os.path.join('/content/drive/MyDrive/testml/Alanine_2/', fileal2[i])
  df = pd.read_csv(filepath, delimiter='\t', header = None)
  df.columns = ['x', 'y']
  scaler = preprocessing.MinMaxScaler()
  names = df.columns
  d = scaler.fit_transform(df)
  df_scaler = pd.DataFrame(d, columns = names)

  deg = 80
  bl = baseline(df_scaler.y, deg = deg)

  wl = 5
  po = 1
  df_filt = savgol_filter(df_scaler.y - bl, window_length = wl, polyorder = po)

  t = {'x': df.x, 'y': df_filt}
  df_final = pd.DataFrame(t, columns = names)
  pd.set_option('display.max_rows', None)
  maindf.at[(i+len(fileal1)), 'y'] = 1
  for j in range(len(arr)):
    maindf.iloc[(i+len(fileal1)), (j+1)] = df_final[df_final['x']==arr[j]].iloc[0, 1]

for i in range(len(fileal3)):
  filepath = os.path.join('/content/drive/MyDrive/testml/Alanine_3/', fileal3[i])
  df = pd.read_csv(filepath, delimiter='\t', header = None)
  df.columns = ['x', 'y']
  scaler = preprocessing.MinMaxScaler()
  names = df.columns
  d = scaler.fit_transform(df)
  df_scaler = pd.DataFrame(d, columns = names)

  deg = 80
  bl = baseline(df_scaler.y, deg = deg)

  wl = 5
  po = 1
  df_filt = savgol_filter(df_scaler.y - bl, window_length = wl, polyorder = po)

  t = {'x': df.x, 'y': df_filt}
  df_final = pd.DataFrame(t, columns = names)
  pd.set_option('display.max_rows', None)
  maindf.at[(i+len(fileal1)+len(fileal2)), 'y'] = 1
  for j in range(len(arr)):
    maindf.iloc[(i+len(fileal1)+len(fileal2)), (j+1)] = df_final[df_final['x']==arr[j]].iloc[0, 1]

for i in range(len(filegl1)):
  filepath = os.path.join('/content/drive/MyDrive/testml/Glutamine_1/', filegl1[i])
  df = pd.read_csv(filepath, delimiter='\t', header = None)
  df.columns = ['x', 'y']
  scaler = preprocessing.MinMaxScaler()
  names = df.columns
  d = scaler.fit_transform(df)
  df_scaler = pd.DataFrame(d, columns = names)

  deg = 50
  bl = baseline(df_scaler.y, deg = deg)

  wl = 5
  po = 1
  df_filt = savgol_filter(df_scaler.y - bl, window_length = wl, polyorder = po)

  t = {'x': df.x, 'y': df_filt}
  df_final = pd.DataFrame(t, columns = names)
  pd.set_option('display.max_rows', None)
  maindf.at[(i+len(fileal1)+len(fileal2)+len(fileal3)), 'y'] = 0
  for j in range(len(arr)):
    maindf.iloc[(i+len(fileal1)+len(fileal2)+len(fileal3)), (j+1)] = df_final[df_final['x']==arr[j]].iloc[0, 1]

for i in range(len(filegl2)):
  filepath = os.path.join('/content/drive/MyDrive/testml/Glutamine_2/', filegl2[i])
  df = pd.read_csv(filepath, delimiter='\t', header = None)
  df.columns = ['x', 'y']
  scaler = preprocessing.MinMaxScaler()
  names = df.columns
  d = scaler.fit_transform(df)
  df_scaler = pd.DataFrame(d, columns = names)

  deg = 50
  bl = baseline(df_scaler.y, deg = deg)

  wl = 5
  po = 1
  df_filt = savgol_filter(df_scaler.y - bl, window_length = wl, polyorder = po)

  t = {'x': df.x, 'y': df_filt}
  df_final = pd.DataFrame(t, columns = names)
  pd.set_option('display.max_rows', None)
  maindf.at[(i+len(fileal1)+len(fileal2)+len(fileal3)+len(filegl1)), 'y'] = 0
  for j in range(len(arr)):
    maindf.iloc[(i+len(fileal1)+len(fileal2)+len(fileal3)+len(filegl1)), (j+1)] = df_final[df_final['x']==arr[j]].iloc[0, 1]

for i in range(len(filegl3)):
  filepath = os.path.join('/content/drive/MyDrive/testml/Glutamine_3/', filegl3[i])
  df = pd.read_csv(filepath, delimiter='\t', header = None)
  df.columns = ['x', 'y']
  scaler = preprocessing.MinMaxScaler()
  names = df.columns
  d = scaler.fit_transform(df)
  df_scaler = pd.DataFrame(d, columns = names)

  deg = 50
  bl = baseline(df_scaler.y, deg = deg)

  wl = 5
  po = 1
  df_filt = savgol_filter(df_scaler.y - bl, window_length = wl, polyorder = po)

  t = {'x': df.x, 'y': df_filt}
  df_final = pd.DataFrame(t, columns = names)
  pd.set_option('display.max_rows', None)
  maindf.at[(i+len(fileal1)+len(fileal2)+len(fileal3)+len(filegl1)+len(filegl2)), 'y'] = 0
  for j in range(len(arr)):
    maindf.iloc[(i+len(fileal1)+len(fileal2)+len(fileal3)+len(filegl1)+len(filegl2)), (j+1)] = df_final[df_final['x']==arr[j]].iloc[0, 1]

# print(maindf)

In [11]:
# разделяю данные на тренировочные и тестовые
x = maindf.iloc[:, 1:].values.tolist()
y = maindf['y'].tolist()

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.8, shuffle = True, stratify = y, random_state=42)

In [12]:
# классификатор DecisionTreeClassifier()
decision_tree = DecisionTreeClassifier()

decision_tree.fit(x_train, y_train)

predicted = decision_tree.predict(x_test)
# print(predicted)
# print(y_test)
for i in range(len(y_test)):
  if y_test[i] != predicted[i]:
    print(predicted[i], i)

0 186
0 240
0 406


In [ ]:
# классификатор RandomForestClassifier()
forest = RandomForestClassifier()

forest.fit(x_train, y_train)

predicted = forest.predict(x_test)
# print(predicted)
# print(y_test)
for i in range(len(y_test)):
  if y_test[i] != predicted[i]:
    print(predicted[i], i)

In [13]:
# значения точности
accuracy = accuracy_score(y_test, predicted)
print(accuracy)

precision = precision_score(y_test, predicted)
print(precision)

recall = recall_score(y_test, predicted)
print(recall)

0.9948364888123924
1.0
0.9896551724137931


In [21]:
# подбор лучших параметров
parameter_grid = {
    'max_depth' : np.arange(1,20,1),
    'min_samples_leaf': np.arange(1,5,1)
}

grid_searcher = GridSearchCV(estimator=DecisionTreeClassifier(),
                             param_grid=parameter_grid,
                             cv=25,
                             scoring='accuracy',
                             n_jobs=-1
                            )

grid_searcher.fit(x_train, y_train)

print(grid_searcher.best_params_)
print(grid_searcher.best_score_)

grid_searcher = GridSearchCV(estimator=DecisionTreeClassifier(),
                             param_grid=parameter_grid,
                             cv=25,
                             scoring='precision',
                             n_jobs=-1
                            )

grid_searcher.fit(x_train, y_train)

print(grid_searcher.best_params_)
print(grid_searcher.best_score_)

grid_searcher = GridSearchCV(estimator=DecisionTreeClassifier(),
                             param_grid=parameter_grid,
                             cv=25,
                             scoring='recall',
                             n_jobs=-1
                            )

grid_searcher.fit(x_train, y_train)

print(grid_searcher.best_params_)
print(grid_searcher.best_score_)

{'max_depth': 1, 'min_samples_leaf': 1}
1.0
{'max_depth': 1, 'min_samples_leaf': 1}
1.0
{'max_depth': 1, 'min_samples_leaf': 1}
1.0


In [23]:
# кросс-валидация
cv = KFold(n_splits = 25, shuffle = True)

scores = cross_validate(decision_tree, x, y, cv=cv, scoring = {'accuracy':'accuracy'}, n_jobs = -1)
print(scores['test_accuracy'])

scores = cross_validate(decision_tree, x, y, cv=cv, scoring = {'precision':'precision'}, n_jobs = -1)
print(scores['test_precision'])

scores = cross_validate(decision_tree, x, y, cv=cv, scoring = {'recall':'recall'}, n_jobs = -1)
print(scores['test_recall'])

[1.         1.         1.         0.96551724 1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.        ]
[1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         0.92307692 1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.        ]
[1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         0.94117647 1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.        ]
